In [2]:
#!/usr/bin/python
#coding:utf-8
import pickle
from flask import Flask,jsonify,Response,request
import os
import logging
import pandas as pd
import pymysql
from sklearn.preprocessing import StandardScaler
import numpy as np
# from sklearn.externals import joblib
import joblib
from tornado.wsgi import WSGIContainer
from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop

app = Flask(__name__)


logger = logging.getLogger('Model')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

hostname = "127.0.0.1"
port = 3306
username = "root"
pwd = "xiaoxiong94"
database = "milk_project"

class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class LostModel(metaclass=Singleton):
    def __init__(self):
        self.initVariable()

    def initVariable(self):
        self.conn = pymysql.connect(host=hostname, port=int(port), user=username, passwd=pwd,db=database,charset='utf8')
        self.clf = joblib.load('model/rf_model.pkl')
        # 加载标准化器
        self.scaler = joblib.load('model/scaler.pkl')

    def detectIsLost(self,uid):
        if uid == None: return jsonify({'result': '-1'})
        if len(uid)<1: return jsonify({'result': '-2'})
        
        newdf = pd.read_sql("select UserId,ActivityCount,ActivityKeep,ActivityUsed,"
                            "IntegralCurrentPoints,IntegralUsed,IngegralTotal,IntegralFrequency,"
                            "IntegralAvgPointsDay,IntegralScanCount,IntegralScanTotal,IntegralScanFrequency,"
                            "IntegralCheckinCount,IntegralCheckinFrequency,OrderCount,"
                            "OrderFrequency,OrderItemCount,OrderAvgPrice,OrderProvinceCity,"
                            "OrderAvgPoint,BabyCount,User_iCreator,ClientCode,LoyaltyIsAutoLost from use_to_predict where userid="+uid, con=self.conn)
                                    # ActivityCount	ActivityKeep	ActivityUsed	IntegralCurrentPoints	IntegralUsed	IngegralTotal	IntegralFrequency	IntegralAvgPointsDay	IntegralScanCount	IntegralScanTotal	IntegralScanFrequency	IntegralCheckinCount	IntegralCheckinFrequency	OrderCount	OrderFrequency	OrderItemCount	OrderAvgPrice	OrderProvinceCity	OrderAvgPoint	BabyCount	User_iCreator	ClientCode
        if newdf.shape[0] <=0 : return jsonify({'result': '-3'})

        # newdf = newdf.fillna(0)
        newdf.ActivityKeep = newdf.ActivityKeep.fillna(0)
        newdf.IntegralCurrentPoints = newdf.IntegralCurrentPoints.fillna(0)
        newdf.IntegralUsed = newdf.IntegralUsed.fillna(0)
        newdf.IntegralCheckinFrequency = newdf.IntegralCheckinFrequency.fillna(0)
        newdf.BabyCount = newdf.BabyCount.fillna(0)
        newdf.User_iCreator = newdf.User_iCreator.fillna(0)
        newdf.ActivityUsed = newdf.ActivityUsed.fillna(0)
        newdf.OrderFrequency = newdf.OrderFrequency.fillna(0)
        newdf.OrderFrequency = newdf.OrderFrequency.fillna(0)
        newdf.ClientCode = newdf.ClientCode.fillna(0)
        factor = pd.factorize(newdf['OrderProvinceCity'])
        newdf.OrderProvinceCity = factor[0]
        newdf = newdf[[
                # 'UserId',
                 'ActivityCount',
                 'ActivityKeep',
                 'ActivityUsed',
                 'IntegralCurrentPoints',
                 'IntegralUsed',
                 'IngegralTotal',
                 'IntegralFrequency',
                 'IntegralAvgPointsDay',
                 'IntegralScanCount',
                 'IntegralScanTotal',
                 'IntegralScanFrequency',
                 'IntegralCheckinCount',
                 'IntegralCheckinFrequency',
                 'OrderCount',
                 'OrderFrequency',
                 'OrderItemCount',
                 'OrderAvgPrice',
                 'OrderProvinceCity',
                 'OrderAvgPoint',
                 'BabyCount',
                 'User_iCreator',
                 'ClientCode',
                #  'MemberState',
                    'LoyaltyIsAutoLost']]
        print("newdf",newdf)
        X = newdf.iloc[:,0:len(newdf.columns.tolist())-1].values
        print(X)
        # 在预测代码中加载平均值和标准差，并使用其对新数据进行标准化

        x_input = self.scaler.transform(X)
        y_pred = self.clf.predict(x_input)
        print(y_pred)
        return jsonify({'result': str(y_pred)})

@app.route('/api/v1.0/islost/<uid>', methods=['GET'])
def detectIsLost(uid):
    tager = LostModel()
    return tager.detectIsLost(uid)

if __name__ == '__main__':
    tager = LostModel()
    http_server = HTTPServer(WSGIContainer(app))
    http_server.listen(24236)
    # 通常每个套接字地址(协议/网络地址/端口)只允许使用一次。决方法是修改程序使用的端口号，例如5000改为8080
    try:
        IOLoop.instance().start()
        tager = LostModel()
    except RuntimeError as e:
        if "This event loop is already running" not in str(e):
            raise




C:\Users\15135\AppData\Local\Temp\ipykernel_27420\1940744086.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  newdf = pd.read_sql("select UserId,ActivityCount,ActivityKeep,ActivityUsed,"


newdf    ActivityCount  ActivityKeep  ActivityUsed  IntegralCurrentPoints   
0              0             0             0                      0  \

   IntegralUsed  IngegralTotal  IntegralFrequency  IntegralAvgPointsDay   
0             0           3060                0.0               4.78125  \

   IntegralScanCount  IntegralScanTotal  ...  OrderCount  OrderFrequency   
0                 12               3060  ...           1             0.0  \

   OrderItemCount  OrderAvgPrice  OrderProvinceCity  OrderAvgPoint  BabyCount   
0               1              0                 -1           3060        1.0  \

   User_iCreator  ClientCode  LoyaltyIsAutoLost  
0              0      6944.0                  1  

[1 rows x 23 columns]
[[ 0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   3.06000e+03  0.00000e+00  4.78125e+00  1.20000e+01  3.06000e+03
   0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00
   1.00000e+00  0.00000e+00 -1.00000e+00  3.06000e+03  1.000

C:\Users\15135\AppData\Local\Temp\ipykernel_27420\1940744086.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  newdf = pd.read_sql("select UserId,ActivityCount,ActivityKeep,ActivityUsed,"


newdf    ActivityCount  ActivityKeep  ActivityUsed  IntegralCurrentPoints   
0              0             0             0                 1072.0  \

   IntegralUsed  IngegralTotal  IntegralFrequency  IntegralAvgPointsDay   
0         288.0           1360              0.013              3.617021  \

   IntegralScanCount  IntegralScanTotal  ...  OrderCount  OrderFrequency   
0                  5               1360  ...           0               0  \

   OrderItemCount  OrderAvgPrice  OrderProvinceCity  OrderAvgPoint  BabyCount   
0               0              0                  0              0        1.0  \

   User_iCreator  ClientCode  LoyaltyIsAutoLost  
0         3877.0     16851.0                  1  

[1 rows x 23 columns]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.07200000e+03
  2.88000000e+02 1.36000000e+03 1.30000000e-02 3.61702128e+00
  5.00000000e+00 1.36000000e+03 1.33333000e+02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e

C:\Users\15135\AppData\Local\Temp\ipykernel_27420\1940744086.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  newdf = pd.read_sql("select UserId,ActivityCount,ActivityKeep,ActivityUsed,"


...

newdf    ActivityCount  ActivityKeep  ActivityUsed  IntegralCurrentPoints   
0              0             0             0                      0  \

   IntegralUsed  IngegralTotal  IntegralFrequency  IntegralAvgPointsDay   
0             0           7670              0.671             13.970856  \

   IntegralScanCount  IntegralScanTotal  ...  OrderCount  OrderFrequency   
0                 57               7670  ...           2             0.0  \

   OrderItemCount  OrderAvgPrice  OrderProvinceCity  OrderAvgPoint  BabyCount   
0               2              0                 -1           4746        1.0  \

   User_iCreator  ClientCode  LoyaltyIsAutoLost  
0         2125.0      9921.0                  1  

[1 rows x 23 columns]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  7.67000000e+03  6.71000000e-01  1.39708561e+01
   5.70000000e+01  7.67000000e+03  6.70588200e+03  0.00000000e+00
   0.00000000e+00  2.00000000e+00  0.00000000e+00  2.00000000e+

C:\Users\15135\AppData\Local\Temp\ipykernel_27420\1940744086.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  newdf = pd.read_sql("select UserId,ActivityCount,ActivityKeep,ActivityUsed,"


newdf    ActivityCount  ActivityKeep  ActivityUsed  IntegralCurrentPoints   
0              0             0             0                      0  \

   IntegralUsed  IngegralTotal  IntegralFrequency  IntegralAvgPointsDay   
0             0           3336                0.0              1.891156  \

   IntegralScanCount  IntegralScanTotal  ...  OrderCount  OrderFrequency   
0                 12               3336  ...           1             0.0  \

   OrderItemCount  OrderAvgPrice  OrderProvinceCity  OrderAvgPoint  BabyCount   
0               2              0                 -1           3396          0  \

   User_iCreator  ClientCode  LoyaltyIsAutoLost  
0            4.0     11552.0                  1  

[1 rows x 23 columns]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  3.33600000e+03  0.00000000e+00  1.89115646e+00
   1.20000000e+01  3.33600000e+03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  2.00000000e+

C:\Users\15135\AppData\Local\Temp\ipykernel_27420\1940744086.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  newdf = pd.read_sql("select UserId,ActivityCount,ActivityKeep,ActivityUsed,"


newdf    ActivityCount  ActivityKeep  ActivityUsed  IntegralCurrentPoints   
0              0             0          -1.0                  248.0  \

   IntegralUsed  IngegralTotal  IntegralFrequency  IntegralAvgPointsDay   
0           0.0            248                0.0                 248.0  \

   IntegralScanCount  IntegralScanTotal  ...  OrderCount  OrderFrequency   
0                  1                248  ...           0               0  \

   OrderItemCount  OrderAvgPrice  OrderProvinceCity  OrderAvgPoint  BabyCount   
0               0              0                  0              0        1.0  \

   User_iCreator  ClientCode  LoyaltyIsAutoLost  
0         8561.0      3534.0                  0  

[1 rows x 23 columns]
[[ 0.000e+00  0.000e+00 -1.000e+00  2.480e+02  0.000e+00  2.480e+02
   0.000e+00  2.480e+02  1.000e+00  2.480e+02  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  1.000e+00  8.561e+03  3.534e+03]]
[0]
